In [1]:
!pip install psycopg2

     |████████████████████████████████| 389kB 5.8MB/s eta 0:00:01
  Running setup.py install for psycopg2 ... done
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
!pip install pandas

  Using cached https://files.pythonhosted.org/packages/47/a5/79156a83c133b5d049a38f444e11eacabab8b3ad00814d8c6811fe9850e2/pandas-1.4.2-cp38-cp38-macosx_10_9_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/64/4a/b008d1f8a7b9f5206ecf70a53f84e654707e7616a771d84c05151a4713e9/numpy-1.22.3.zip
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Using cached https://files.pythonhosted.org/packages/60/2e/dec1cc18c51b8df33c7c4d0a321b084cf38e1733b98f9d15018880fb4970/pytz-2022.1-py2.py3-none-any.whl
  Created wheel for numpy: filename=numpy-1.22.3-cp38-cp38-macosx_10_12_x86_64.whl size=5818944 sha256=18b57fd5e80402860265e87e96b7a36b02293c10044ae61eb9912ecf1b02af66
  Stored in directory: /Users/gaurnitai/Library/Caches/pip/wheels/21/37/10/81a13af5964837a3627457192fbc14d926b1a2dcd810f70fbf
Successfully built numpy
You should consider upgrading via the 'pip install --upgrade pip' command.


In [108]:
import psycopg2 as pg
import pandas as pd 

In [173]:
survey_public_df = pd.read_csv('../data/survey_results_public.csv')

In [174]:
survey_schema_df = pd.read_csv('../data/survey_results_schema.csv')

In [204]:
survey_public_df.shape

(88883, 9)

In [175]:
survey_public_df.head()

,Respondent,MainBranch,Hobbyist,OpenSourcer,OpenSource,Employment,Country,Student,EdLevel,UndergradMajor,...,WelcomeChange,SONewContent,Age,Gender,Trans,Sexuality,Ethnicity,Dependents,SurveyLength,SurveyEase
0,1,I am a student who is learning to code,Yes,Never,The quality of OSS and closed source software ...,"Not employed, and not looking for work",United Kingdom,No,Primary/elementary school,NaN,...,Just as welcome now as I felt last year,Tech articles written by other developers;Indu...,14.0,Man,No,Straight / Heterosexual,NaN,No,Appropriate in length,Neither easy nor difficult
1,2,I am a student who is learning to code,No,Less than once per year,The quality of OSS and closed source software ...,"Not employed, but looking for work",Bosnia and Herzegovina,"Yes, full-time","Secondary school (e.g. American high school, G...",NaN,...,Just as welcome now as I felt last year,Tech articles written by other developers;Indu...,19.0,Man,No,Straight / Heterosexual,NaN,No,Appropriate in length,Neither easy nor difficult
2,3,"I am not primarily a developer, but I write co...",Yes,Never,The quality of OSS and closed source software ...,Employed full-time,Thailand,No,"Bachelor’s degree (BA, BS, B.Eng., etc.)",Web development or web design,...,Just as welcome now as I felt last year,Tech meetups or events in your area;Courses on...,28.0,Man,No,Straight / Heterosexual,NaN,Yes,Appropriate in length,Neither easy nor difficult
3,4,I am a developer by profession,No,Never,The quality of OSS and closed source software ...,Employed full-time,United States,No,"Bachelor’s degree (BA, BS, B.Eng., etc.)","Computer science, computer engineering, or sof...",...,Just as welcome now as I felt last year,Tech articles written by other developers;Indu...,22.0,Man,No,Straight / Heterosexual,White or of European descent,No,Appropriate in length,Easy
4,5,I am a developer by profession,Yes,Once a month or more often,"OSS is, on average, of HIGHER quality than pro...",Employed full-time,Ukraine,No,"Bachelor’s degree (BA, BS, B.Eng., etc.)","Computer science, computer engineering, or sof...",...,Just as welcome now as I felt last year,Tech meetups or events in your area;Courses on...,30.0,Man,No,Straight / Heterosexual,White or of European descent;Multiracial,No,Appropriate in length,Easy


In [176]:
survey_schema_df.head()

,Column,QuestionText
0,Respondent,Randomized respondent ID number (not in order ...
1,MainBranch,Which of the following options best describes ...
2,Hobbyist,Do you code as a hobby?
3,OpenSourcer,How often do you contribute to open source?
4,OpenSource,How do you feel about the quality of open sour...


In [177]:
# Lets drop some of the survey_schema_df columns
columns_to_delete = survey_public_df.loc[:, 'UndergradMajor':]

In [178]:
survey_public_df.drop(columns=columns_to_delete, axis=1, inplace=True)

In [179]:
survey_public_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88883 entries, 0 to 88882
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Respondent   88883 non-null  int64 
 1   MainBranch   88331 non-null  object
 2   Hobbyist     88883 non-null  object
 3   OpenSourcer  88883 non-null  object
 4   OpenSource   86842 non-null  object
 5   Employment   87181 non-null  object
 6   Country      88751 non-null  object
 7   Student      87014 non-null  object
 8   EdLevel      86390 non-null  object
dtypes: int64(1), object(8)
memory usage: 6.1+ MB


In [180]:
#conn = pg.connect(host=hostname, port=port, user=username, password=password)

In [310]:
# Lets wrap in try except to handle any connection error
def connect_to_db(hostname, port, username, password):
    try:
        conn = pg.connect(host=hostname, port=port, user=username, password=password)
    except OperationError as e:
        raise e
    else:
        print('Connected Successfully....')
    return conn

In [311]:
hostname='localhost'
port='5432'
username='surveyuser'
password='superuser'
database='survey'
conn=None

In [312]:
conn = connect_to_db(hostname, port, username, password)

Connected Successfully....


In [313]:
# Now lets create cursor to run sql query and fetch the result from database
curr = conn.cursor()

In [314]:
# Get column names from csv
survey_public_df.columns

Index(['respondent', 'mainbranch', 'hobbyist', 'opensourcer', 'opensource',
       'employment', 'country', 'student', 'edlevel'],
      dtype='object')

In [315]:
survey_public_df.loc[1:10, :]

,respondent,mainbranch,hobbyist,opensourcer,opensource,employment,country,student,edlevel
1,2,I am a student who is learning to code,No,Less than once per year,The quality of OSS and closed source software ...,"Not employed, but looking for work",Bosnia and Herzegovina,"Yes, full-time","Secondary school (e.g. American high school, G..."
2,3,"I am not primarily a developer, but I write co...",Yes,Never,The quality of OSS and closed source software ...,Employed full-time,Thailand,No,"Bachelor’s degree (BA, BS, B.Eng., etc.)"
3,4,I am a developer by profession,No,Never,The quality of OSS and closed source software ...,Employed full-time,United States,No,"Bachelor’s degree (BA, BS, B.Eng., etc.)"
4,5,I am a developer by profession,Yes,Once a month or more often,"OSS is, on average, of HIGHER quality than pro...",Employed full-time,Ukraine,No,"Bachelor’s degree (BA, BS, B.Eng., etc.)"
5,6,"I am not primarily a developer, but I write co...",Yes,Never,The quality of OSS and closed source software ...,Employed full-time,Canada,No,"Bachelor’s degree (BA, BS, B.Eng., etc.)"
6,7,I am a developer by profession,No,Never,The quality of OSS and closed source software ...,"Independent contractor, freelancer, or self-em...",Ukraine,No,"Bachelor’s degree (BA, BS, B.Eng., etc.)"
7,8,I code primarily as a hobby,Yes,Less than once per year,"OSS is, on average, of HIGHER quality than pro...","Not employed, but looking for work",India,NaN,"Bachelor’s degree (BA, BS, B.Eng., etc.)"
8,9,I am a developer by profession,Yes,Once a month or more often,The quality of OSS and closed source software ...,Employed full-time,New Zealand,No,Some college/university study without earning ...
9,10,I am a developer by profession,Yes,Once a month or more often,"OSS is, on average, of HIGHER quality than pro...",Employed full-time,India,No,"Master’s degree (MA, MS, M.Eng., MBA, etc.)"
10,11,I code primarily as a hobby,Yes,Once a month or more often,The quality of OSS and closed source software ...,NaN,Antigua and Barbuda,"Yes, full-time",Primary/elementary school


In [316]:
def rename_columns(column):
    col = column.replace(' ','')
    return col.lower()

In [317]:
survey_public_df.columns.map(rename_columns)

Index(['respondent', 'mainbranch', 'hobbyist', 'opensourcer', 'opensource',
       'employment', 'country', 'student', 'edlevel'],
      dtype='object')

In [318]:
survey_public_df.columns = survey_public_df.columns.map(rename_columns)
survey_public_df.columns

Index(['respondent', 'mainbranch', 'hobbyist', 'opensourcer', 'opensource',
       'employment', 'country', 'student', 'edlevel'],
      dtype='object')

In [349]:
# CREATE TABLE to export csv data to table
def create_table(curr):
    create_table_command = ('''
        CREATE TABLE IF NOT EXISTS survey_public (
        respondent INTEGER PRIMARY KEY, 
        mainbranch VARCHAR(255) NOT NULL, 
        hobbyist VARCHAR(255) NOT NULL, 
        opensourcer VARCHAR(255) NULL,  
        opensource VARCHAR(255) NULL,
        employment VARCHAR(255) NULL, 
        country VARCHAR(255) NULL, 
        student VARCHAR(255) NULL, 
        edlevel VARCHAR(255) NULL
        )
    ''')
    curr.execute(create_table_command)

In [350]:
create_table(curr)
conn.commit()

In [351]:
# Now lets export data from csv to database
# Here are two use cases possible
# 1. Record may already exist - Update operation
# 2. New record need to be enter - Insert operation
# for i, row in survey_public_df.iterrows():
#     if check_if_data_exists():
#         update_row()
#     else:
#         insert_row()

In [352]:
def check_if_data_exists(curr, respondent):
    query = ("""SELECT respondent FROM survey_public WHERE respondent = %s""")
    curr.execute(query, (respondent,))
    return curr.fetchone() is not None
    

In [358]:
def update_row(curr, respondent, mainbranch, hobbyist, opensourcer, 
               opensource,employment, country, student, edlevel):
    query = ("""UPDATE survey_public 
    SET mainbranch= %s,
        hobbyist= %s,
        opensourcer = %s,
        opensource = %s,
        employment = %s,
        country = %s,
        student = %s,
        edlevel = %s 
        WHERE respondent= %s;
    """)
    row_to_update = (mainbranch, hobbyist, opensourcer, 
               opensource,employment, country, student, edlevel, respondent)
    curr.execute(query, row_to_update)

In [359]:
def insert_row(curr, respondent, mainbranch, hobbyist, opensourcer, 
               opensource,employment, country, student, edlevel):
    query = ("""INSERT INTO survey_public (respondent, mainbranch, hobbyist, opensourcer, 
               opensource,employment, country, student, edlevel)
               VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s);""")
    row_to_insert = (respondent, mainbranch, hobbyist, opensourcer, 
               opensource,employment, country, student, edlevel)
    curr.execute(query, row_to_insert)

In [360]:
# Now insering data from df to db
def insert_data_from_df_to_db(curr, df):
    for i, row in df.iterrows():
        insert_row(curr, row['respondent'], row['mainbranch'], row['hobbyist'],
                   row['opensourcer'], row['opensource'], row['employment'],
                   row['country'], row['student'], row['edlevel'])
    

In [361]:
# Here we will iter over df abd find whats the updated record and new record and 
# from there we will create another df for new records
def update_data_from_df_to_db(curr, df):
    # create blank temp df
    temp_df = pd.DataFrame(columns=['respondent', 'mainbranch', 'hobbyist', 'opensourcer', 'opensource',
       'employment', 'country', 'student', 'edlevel'])
    
    for i, row in df.iterrows():
        print(row)
        print(check_if_data_exists(curr, row['respondent']))
        if check_if_data_exists(curr, row['respondent']):
            print('Updating record....')
            update_row(curr,row['respondent'], row['mainbranch'], row['hobbyist'],
                   row['opensourcer'], row['opensource'], row['employment'],
                   row['country'], row['student'], row['edlevel'])
        else:
            print('Inserting record....')
            temp_df = temp_df.append(row)
    print(temp_df)
    return temp_df

In [362]:
survey_public_df.loc[80:120, :].head(2)

,respondent,mainbranch,hobbyist,opensourcer,opensource,employment,country,student,edlevel
80,81,I am a developer by profession,Yes,Less than once a month but more than once per ...,The quality of OSS and closed source software ...,Employed full-time,Poland,No,"Master’s degree (MA, MS, M.Eng., MBA, etc.)"
81,82,I am a developer by profession,Yes,Once a month or more often,"OSS is, on average, of HIGHER quality than pro...",Employed full-time,United Kingdom,No,"Master’s degree (MA, MS, M.Eng., MBA, etc.)"


In [369]:
new_survey_data_df = update_data_from_df_to_db(curr, survey_public_df.loc[95:1000,:])
conn.commit()

respondent                                                    96
mainbranch     I used to be a developer by profession, but no...
hobbyist                                                      No
opensourcer                                                Never
opensource     The quality of OSS and closed source software ...
employment                                    Employed full-time
country                                                   Canada
student                                                       No
edlevel                 Bachelor’s degree (BA, BS, B.Eng., etc.)
Name: 95, dtype: object
True
Updating record....
respondent                                                    97
mainbranch     I am not primarily a developer, but I write co...
hobbyist                                                     Yes
opensourcer                                                Never
opensource     The quality of OSS and closed source software ...
employment                               

/var/folders/l_/7k2bzgjj6kncx_c0wt_ckd4m0000gn/T/ipykernel_76682/2879580461.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp_df = temp_df.append(row)
/var/folders/l_/7k2bzgjj6kncx_c0wt_ckd4m0000gn/T/ipykernel_76682/2879580461.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp_df = temp_df.append(row)
/var/folders/l_/7k2bzgjj6kncx_c0wt_ckd4m0000gn/T/ipykernel_76682/2879580461.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp_df = temp_df.append(row)
/var/folders/l_/7k2bzgjj6kncx_c0wt_ckd4m0000gn/T/ipykernel_76682/2879580461.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp_df = temp_df.append(row)
/var/fol

False
Inserting record....
respondent                                                   118
mainbranch                        I am a developer by profession
hobbyist                                                     Yes
opensourcer                           Once a month or more often
opensource     OSS is, on average, of HIGHER quality than pro...
employment                                    Employed full-time
country                                                    Italy
student                                                       No
edlevel                 Bachelor’s degree (BA, BS, B.Eng., etc.)
Name: 117, dtype: object
False
Inserting record....
respondent                                                   119
mainbranch                        I am a developer by profession
hobbyist                                                      No
opensourcer                              Less than once per year
opensource     The quality of OSS and closed source software ...
employment 

/var/folders/l_/7k2bzgjj6kncx_c0wt_ckd4m0000gn/T/ipykernel_76682/2879580461.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp_df = temp_df.append(row)
/var/folders/l_/7k2bzgjj6kncx_c0wt_ckd4m0000gn/T/ipykernel_76682/2879580461.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp_df = temp_df.append(row)
/var/folders/l_/7k2bzgjj6kncx_c0wt_ckd4m0000gn/T/ipykernel_76682/2879580461.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp_df = temp_df.append(row)
/var/folders/l_/7k2bzgjj6kncx_c0wt_ckd4m0000gn/T/ipykernel_76682/2879580461.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp_df = temp_df.append(row)
/var/fol

respondent                                                   129
mainbranch     I am not primarily a developer, but I write co...
hobbyist                                                     Yes
opensourcer                                                Never
opensource     OSS is, on average, of HIGHER quality than pro...
employment                                    Employed full-time
country                                            United States
student                                                       No
edlevel                Other doctoral degree (Ph.D, Ed.D., etc.)
Name: 128, dtype: object
False
Inserting record....
respondent                                                   130
mainbranch                        I am a developer by profession
hobbyist                                                     Yes
opensourcer                                                Never
opensource     The quality of OSS and closed source software ...
employment                            

...

False
Inserting record....
respondent                                                   978
mainbranch                        I am a developer by profession
hobbyist                                                     Yes
opensourcer                              Less than once per year
opensource     The quality of OSS and closed source software ...
employment                                    Employed full-time
country                                                  Ukraine
student                                                       No
edlevel              Master’s degree (MA, MS, M.Eng., MBA, etc.)
Name: 974, dtype: object
False
Inserting record....
respondent                                                   979
mainbranch     I am not primarily a developer, but I write co...
hobbyist                                                      No
opensourcer                              Less than once per year
opensource     The quality of OSS and closed source software ...
employment 

/var/folders/l_/7k2bzgjj6kncx_c0wt_ckd4m0000gn/T/ipykernel_76682/2879580461.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp_df = temp_df.append(row)
/var/folders/l_/7k2bzgjj6kncx_c0wt_ckd4m0000gn/T/ipykernel_76682/2879580461.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp_df = temp_df.append(row)
/var/folders/l_/7k2bzgjj6kncx_c0wt_ckd4m0000gn/T/ipykernel_76682/2879580461.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp_df = temp_df.append(row)
/var/folders/l_/7k2bzgjj6kncx_c0wt_ckd4m0000gn/T/ipykernel_76682/2879580461.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp_df = temp_df.append(row)
/var/fol

respondent                                                   990
mainbranch     I am not primarily a developer, but I write co...
hobbyist                                                     Yes
opensourcer                                                Never
opensource     OSS is, on average, of HIGHER quality than pro...
employment                    Not employed, but looking for work
country                                            United States
student                                           Yes, part-time
edlevel                 Bachelor’s degree (BA, BS, B.Eng., etc.)
Name: 986, dtype: object
False
Inserting record....
respondent                                                   991
mainbranch                        I am a developer by profession
hobbyist                                                     Yes
opensourcer                              Less than once per year
opensource     OSS is, on average, of HIGHER quality than pro...
employment     Independent contractor,

False
Inserting record....
respondent                                                  1004
mainbranch                        I am a developer by profession
hobbyist                                                     Yes
opensourcer    Less than once a month but more than once per ...
opensource     The quality of OSS and closed source software ...
employment     Independent contractor, freelancer, or self-em...
country                                           United Kingdom
student                                                       No
edlevel                 Bachelor’s degree (BA, BS, B.Eng., etc.)
Name: 1000, dtype: object
False
Inserting record....
     respondent                              mainbranch hobbyist  \
111         112          I am a developer by profession      Yes   
112         113          I am a developer by profession      Yes   
113         114          I am a developer by profession       No   
114         115          I am a developer by profession      Ye

/var/folders/l_/7k2bzgjj6kncx_c0wt_ckd4m0000gn/T/ipykernel_76682/2879580461.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp_df = temp_df.append(row)
/var/folders/l_/7k2bzgjj6kncx_c0wt_ckd4m0000gn/T/ipykernel_76682/2879580461.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp_df = temp_df.append(row)
/var/folders/l_/7k2bzgjj6kncx_c0wt_ckd4m0000gn/T/ipykernel_76682/2879580461.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp_df = temp_df.append(row)
/var/folders/l_/7k2bzgjj6kncx_c0wt_ckd4m0000gn/T/ipykernel_76682/2879580461.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp_df = temp_df.append(row)
/var/fol

In [367]:
new_survey_data_df

,respondent,mainbranch,hobbyist,opensourcer,opensource,employment,country,student,edlevel
101,102,I am a developer by profession,No,Never,The quality of OSS and closed source software ...,Employed full-time,United States,No,"Bachelor’s degree (BA, BS, B.Eng., etc.)"
102,103,I code primarily as a hobby,Yes,Less than once a month but more than once per ...,The quality of OSS and closed source software ...,"Not employed, but looking for work",Russian Federation,No,Associate degree
103,104,I am a developer by profession,Yes,Never,"OSS is, on average, of LOWER quality than prop...",Employed full-time,India,"Yes, full-time","Bachelor’s degree (BA, BS, B.Eng., etc.)"
104,105,I am a developer by profession,Yes,Less than once per year,The quality of OSS and closed source software ...,Employed full-time,United States,No,"Bachelor’s degree (BA, BS, B.Eng., etc.)"
105,106,I am a developer by profession,Yes,Less than once per year,NaN,Employed full-time,United States,No,"Master’s degree (MA, MS, M.Eng., MBA, etc.)"
106,107,I am a developer by profession,Yes,Less than once a month but more than once per ...,The quality of OSS and closed source software ...,Employed part-time,Netherlands,No,"Master’s degree (MA, MS, M.Eng., MBA, etc.)"
107,108,I am a developer by profession,Yes,Less than once per year,"OSS is, on average, of HIGHER quality than pro...",Employed full-time,United States,No,"Bachelor’s degree (BA, BS, B.Eng., etc.)"
108,109,I am a developer by profession,Yes,Never,The quality of OSS and closed source software ...,Employed full-time,Croatia,No,"Master’s degree (MA, MS, M.Eng., MBA, etc.)"
109,110,I am a developer by profession,Yes,Once a month or more often,"OSS is, on average, of HIGHER quality than pro...",Employed full-time,United States,No,"Master’s degree (MA, MS, M.Eng., MBA, etc.)"
110,111,I am a developer by profession,Yes,Never,The quality of OSS and closed source software ...,Employed full-time,United States,No,"Bachelor’s degree (BA, BS, B.Eng., etc.)"


In [370]:
insert_data_from_df_to_db(curr, new_survey_data_df)
conn.commit()